In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import re
import pymorphy2
import joblib
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt

In [3]:
seed = 42
positive_file = "../data/positive.csv"
negative_file = "../data/negative.csv"

In [4]:
np.random.seed(seed)

In [5]:
column_names = ["id", "tdate", "tmane", "ttext", "ttype", "trep", "trtw", "tfav", "tstcount", "tfoll", "tfrien", "listcount"]
positive_df = pd.read_csv(positive_file, sep=";", names=column_names, index_col=False)
negative_df = pd.read_csv(negative_file, sep=";", names=column_names, index_col=False)

In [6]:
# Смена метки класса для отрицательной эмоциональной окраски
negative_df["ttype"] = 0

In [7]:
df = pd.concat([negative_df, positive_df])
df.shape, negative_df.shape, positive_df.shape

((226834, 12), (111923, 12), (114911, 12))

In [8]:
df = df[["ttext", "ttype"]]
df.columns = ['text', 'target']

In [9]:
df["text"].tolist()[:40]

['на работе был полный пиддес :| и так каждое закрытие месяца, я же свихнусь так D:',
 'Коллеги сидят рубятся в Urban terror, а я из-за долбанной винды не могу :(',
 '@elina_4post как говорят обещаного три года ждут...((',
 'Желаю хорошего полёта и удачной посадки,я буду очень сильно скучать( http://t.co/jCLNzVNv3S',
 'Обновил за каким-то лешим surf, теперь не работает простоплеер :(',
 'Котёнка вчера носик разбила, плакала и расстраивалась :(',
 '@juliamayko @O_nika55 @and_Possum Зашли, а то он опять затихарился, я прямо физически страдаю, когда он долго молчит!(((',
 'а вообще я не болею -  я не выздоравливаю :(',
 'я микрофраза :( учимся срать кирпичами в режиме &amp;quot;нон-стоп&amp;quot; @niwoqisipapy',
 'я хочу с тобой помириться , но сука я гордая и никогда этого не сделаю! (((',
 '@DNO_OKEANA_A3A3 @MOE_MOPE_A3A3 тебя ебет какие у меня фотки.я про твои молчу.и вообще ты хоть знаешь как ТП то выглядят...',
 'Блин начали сниться сны( Не когда почти не снились ! А теперь можно ска

In [10]:
def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    # Remove digits
    text = re.sub("\d+:\d+", " ", text)
    text = re.sub(" \d+", " ", text)
    # Removing ;quot; and &amp
    text = re.sub(';quot;', ' ', text) 
    text = re.sub('&amp', ' ', text) 
    # Remove HTML special entities 
    text = re.sub(r'\&\w*;', ' ', text)
    #Convert @username to AT_USER
    text = re.sub('@[^\s]+','at_user', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+', ' ', text)
    # Removing '#' hash tag
    text = re.sub('#', 'hash ', text) 
    # Removing RT
    text = re.sub('rt[\s]+', '', text) 
    # Removing hyperlink
    text = re.sub('https?:\/\/\S+', 'url', text)
    # Separate words and punctuation
    text = re.findall(r"[\w']+|[.,!?;:()]", text)
    text = " ".join(text)
    return text

In [11]:
# Clean the tweets
df['text'] = df['text'].apply(preprocess_text)

In [12]:
cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = []
    for token in text.split():
        # Если токен уже был закеширован, быстро возьмем результат из кэша.
        if token in cache.keys():
            words.append(cache[token])
        # Слово еще не встретилось, будем проводить медленный морфологический анализ.
        else:
            result = morph.parse(token)   
            word = result[0].normal_form
            # Отправляем слово в результат, ...
            words.append(word)
            # ... и кешируем результат его разбора.
            cache[token] = word   
    return ' '.join(words)

In [13]:
%%time

df['text'] = df['text'].apply(lemmatize)

CPU times: user 43.5 s, sys: 100 ms, total: 43.6 s
Wall time: 43.6 s


In [14]:
df["text"].tolist()[:40]

['на работа быть полный пиддес : и так каждый закрытие месяц , я же свихнуться так d :',
 'коллега сидеть рубиться в urban terror , а я из за долбать винд не мочь : (',
 'at_user как говорят обещаной три год ждать . . . ( (',
 'желать хороший полёт и удачный посадка , я быть очень сильно скучать ( url',
 'обновить за какой то леший surf , теперь не работать простоплеер : (',
 'котёнок вчера носик разбить , плакать и расстраиваться : (',
 'at_user at_user at_user заслать , а то он опять затихариться , я прямо физически страдать , когда он долго молчать ! ( ( (',
 'а вообще я не болеть я не выздоравливать : (',
 'я микрофраза : ( учиться срать кирпич в режим нона стоп at_user',
 'я хотеть с ты помириться , но сук я гордый и никогда это не сделать ! ( ( (',
 'at_user at_user ты ебета какой у я фотка . я про твой молчать . и вообще ты хоть знаешь как тп то выглядеть . . .',
 'блин начать сниться сон ( не когда почти не сниться ! а теперь можно сказать каждый день такой ! я это не нравиться

In [15]:
df["target"].tolist()[:40]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df["text"].tolist(), df["target"].tolist(), 
    test_size=0.33, 
    random_state=42, 
    shuffle=True, 
    stratify=df["target"].tolist()
)

In [18]:
def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in tweet.split():
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1    
    return freqs

In [19]:
freqs = build_freqs(X_train, y_train)

In [20]:
freqs[('я', 0)], freqs[('я', 1)]

(39717, 32555)

In [21]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 2)) 
    
    # loop through each word in the list of words
    for word in tweet.split():
        
        # increment the word count for the positive label 1
        x[0,0] += freqs.get((word, 1)) if freqs.get((word, 1)) else 0
        
        # increment the word count for the negative label 0
        x[0,1] += freqs.get((word, 0)) if freqs.get((word, 0)) else 0
        
    assert(x.shape == (1, 2))
    return x

In [22]:
# collect the features 'x' and stack them into a matrix 'X'
X_train_features = np.zeros((len(X_train), 2))
for i in range(len(X_train)):
    X_train_features[i, :]= extract_features(X_train[i], freqs)

In [23]:
X_train_features[:10]

array([[ 269364.,  369324.],
       [ 421216.,  264586.],
       [ 185628.,  208637.],
       [ 350531.,  206837.],
       [ 366061.,   73040.],
       [ 167601.,  282624.],
       [  19351., 1863304.],
       [ 346173.,  312400.],
       [ 783480.,  397093.],
       [ 280892.,  391066.]])

In [24]:
# collect the features 'x' and stack them into a matrix 'X'
X_test_features = np.zeros((len(X_test), 2))
for i in range(len(X_test)):
    X_test_features[i, :]= extract_features(X_test[i], freqs)

In [25]:
pipeline = Pipeline([
    ('classifier', LogisticRegression()),
])

parameters = {
        "classifier": [LogisticRegression()],
        "classifier__penalty": ['l2','l1'],
        "classifier__C": np.logspace(0, 4, 10)
}

In [26]:
grid = GridSearchCV(pipeline, cv=5, param_grid=parameters, verbose=1, n_jobs=20, scoring="f1")
grid.fit(X_train_features, y_train)
print(f"Best Model: {grid.best_score_} using {grid.best_params_}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    2.9s finished


Best Model: 0.9731626948339157 using {'classifier': LogisticRegression(), 'classifier__C': 1.0, 'classifier__penalty': 'l2'}


In [27]:
# save best model to current working directory
joblib.dump(grid, "models/02_sklearn_pipeline_linear_classifier.pkl")
# load from file and predict using the best configs found in the CV step
model = joblib.load("models/02_sklearn_pipeline_linear_classifier.pkl" )
# get predictions from best model above
y_preds = model.predict(X_test_features)
print('accuracy score: ', accuracy_score(y_test, y_preds))
print('\n')
print('confusion matrix: \n', confusion_matrix(y_test,y_preds))
print('\n')
print(classification_report(y_test, y_preds))

accuracy score:  0.9724003419899541


confusion matrix: 
 [[35335  1600]
 [  466 37455]]


              precision    recall  f1-score   support

           0       0.99      0.96      0.97     36935
           1       0.96      0.99      0.97     37921

    accuracy                           0.97     74856
   macro avg       0.97      0.97      0.97     74856
weighted avg       0.97      0.97      0.97     74856

